# **ALCALDES**

De los .XSLX sacados del gobierno se leerá los ficheros de una carpeta XSLX.

Y se añadirá en una nueva columna los nombres limpiados en mayúsculas

In [89]:
import pathlib
XSLX=str(pathlib.Path().resolve())+"/XSLX"
from os import listdir
from os.path import isfile, join

ficherosXSLX = [f for f in listdir(XSLX) if isfile(join(XSLX, f))] #Tenemos en la variable ficherosXSLX la lista de ficheros de la carpeta

In [90]:
import pandas as pd


def cargaAlcaldes(archivo, columna, fila, cabecera, nombreHoja):
    return pd.read_excel(archivo, sheet_name=nombreHoja,
         names=cabecera, usecols=columna, skiprows=fila)

dfAlcaldes = pd.DataFrame([])

for fichero in ficherosXSLX:
    """
    Con los ficheros leidos en el paso anterior se leerá de 1 en 1 de la lista con el método de cargaAlcaldes
    """
    print(fichero)
    dfAlcaldes = pd.concat([dfAlcaldes,
                            cargaAlcaldes(XSLX+"/"+fichero,"D:K",8,
        ['CCAA', 'PROVINCIA', 'MUNICIPIO', 'CODIGO INE', 'NOMBRE','LISTA','FECHA POSESIÓN','FECHA BAJA'], 
        pd.ExcelFile(XSLX+"/"+fichero).sheet_names[0])],)




Alcaldes_Mandato_1979_1983.xlsx
Alcaldes_Mandato_1983_1987.xlsx
Alcaldes_Mandato_1987_1991.xlsx
Alcaldes_Mandato_1991_1995.xlsx
Alcaldes_Mandato_1995_1999.xlsx
Alcaldes_Mandato_1999_2003.xlsx
Alcaldes_Mandato_2003_2007.xlsx
Alcaldes_Mandato_2007_2011.xlsx
Alcaldes_Mandato_2011_2015.xlsx
Alcaldes_Mandato_2015_2019.xlsx
Alcaldes_Mandato_2019_2023.xlsx


**Limpieza de datos**

In [91]:
import re
import string
import unicodedata

def transformar_fecha(columna,dataFrame):
    dataFrame[columna]=(dataFrame[columna].astype(str))
    dataFrame.loc[dataFrame[columna] == 'SIN FECHA', columna] = pd.NA
    dataFrame[columna]=pd.to_datetime((dataFrame[columna].str.replace("-","/").str[:10]),format='mixed')
    return dataFrame

"""
    Remplaza todos los signos de puntuación por vacío eliminando '.,'...
    Cambia la normalización con el encode ASCII y lo descodifica a UTF-8 español para eliminar todas las tildes
    Convierte todo a UPPER para que los nombres estén en mayúscula
"""
dfAlcaldes['NOMBRE NORMALIZADO']=(dfAlcaldes['NOMBRE']
                                  .str.replace('[{}]'.format(string.punctuation), '',regex=True)
                                  .str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper())


dfAlcaldes = transformar_fecha("FECHA POSESIÓN",dfAlcaldes)
dfAlcaldes = transformar_fecha("FECHA BAJA",dfAlcaldes)



In [92]:
dfAlcaldes

,CCAA,PROVINCIA,MUNICIPIO,CODIGO INE,NOMBRE,LISTA,FECHA POSESIÓN,FECHA BAJA,NOMBRE NORMALIZADO
0,Andalucía,Almería,ABRUCENA,040025,"MARTINEZ LAO, JUAN",PSOE,1979-04-19,NaT,MARTINEZ LAO JUAN
1,Andalucía,Almería,ADRA,040031,"SARMIENTO POSADA, PEDRO",OTROS,1979-04-19,NaT,SARMIENTO POSADA PEDRO
2,Andalucía,Almería,ALBANCHEZ,040046,"MARTOS AYBAR, RAMON",UCD,1979-04-19,NaT,MARTOS AYBAR RAMON
3,Andalucía,Almería,ALBOLODUY,040059,"BLANES PANIAGUA, MARIANO",UCD,1979-04-19,NaT,BLANES PANIAGUA MARIANO
4,Andalucía,Almería,ALBOX,040062,"MIRAS CARRASCO, JOSE",UCD,1979-04-19,NaT,MIRAS CARRASCO JOSE
...,...,...,...,...,...,...,...,...,...
8758,"Rioja, La","Rioja, La",Viniegra de Abajo,261780,Juan Ignacio Martínez Fernández,PP,2019-06-15,NaT,JUAN IGNACIO MARTINEZ FERNANDEZ
8759,"Rioja, La","Rioja, La",Viniegra de Arriba,261793,Laura Crespi Lázaro,PP,2019-06-15,NaT,LAURA CRESPI LAZARO
8760,"Rioja, La","Rioja, La",Zarratón,261807,Víctor Manuel Manso Ocio,PSOE,2019-06-15,NaT,VICTOR MANUEL MANSO OCIO
8761,"Rioja, La","Rioja, La",Zarzosa,261814,José María Sáenz Blanco,PP,2019-06-15,NaT,JOSE MARIA SAENZ BLANCO


**ALCALDES QUE REPITIERON ALCALDÍA**

In [ ]:
df_duplicated = dfAlcaldes[dfAlcaldes.duplicated(subset='NOMBRE NORMALIZADO', keep=False)]
# Contar el número de filas duplicadas por cada valor en 'NOMBRE NORMALIZADO'
duplicated_counts = df_duplicated.groupby('NOMBRE NORMALIZADO').size()

# Añadir la cuenta de duplicados como una nueva columna en df_duplicated
df_duplicated['DUPLICADOS'] = df_duplicated['NOMBRE NORMALIZADO'].map(duplicated_counts)

**CUANDO FUE SU PRIMERA ALCALDÍA**

In [ ]:
min_dates = df_duplicated.groupby('NOMBRE NORMALIZADO')['FECHA POSESIÓN'].min()

df_duplicated['PRIMERA ALCALDÍA'] = df_duplicated['NOMBRE NORMALIZADO'].map(min_dates)

df_duplicated

**ÚLTIMA ALCALDÍA**

In [ ]:
min_dates = df_duplicated.groupby('NOMBRE NORMALIZADO')['FECHA POSESIÓN'].max()

df_duplicated['COMIENZO ÚLTIMA ALCALDÍA'] = df_duplicated['NOMBRE NORMALIZADO'].map(min_dates)

df_duplicated

**VECES DE BAJA**

In [ ]:
df_duplicated['VECES F.BAJA'] = df_duplicated['FECHA BAJA'].notna().astype(int)

min_dates = df_duplicated.groupby('NOMBRE NORMALIZADO')['VECES F.BAJA'].sum()

df_duplicated['VECES F.BAJA'] = df_duplicated['NOMBRE NORMALIZADO'].map(min_dates)

df_duplicated

**CREACIÓN DATAFRAME CON DATOS LIMPIOS**

In [99]:
dfAlcaldesLimpio=pd.DataFrame()
dfAlcaldesLimpio['NOMBRE LIMPIO'] = df_duplicated['NOMBRE NORMALIZADO']
dfAlcaldesLimpio['NUM ALCALDÍAS'] = df_duplicated['DUPLICADOS']
dfAlcaldesLimpio['PRIMERA ALCALDÍA'] = df_duplicated['PRIMERA ALCALDÍA']
dfAlcaldesLimpio['ÚLTIMA ALCALDÍA'] = df_duplicated['COMIENZO ÚLTIMA ALCALDÍA']
dfAlcaldesLimpio['VECES F.BAJA'] = df_duplicated['VECES F.BAJA']

dfAlcaldesLimpio

,NOMBRE LIMPIO,NUM ALCALDÍAS,PRIMERA ALCALDÍA,ÚLTIMA ALCALDÍA,VECES F.BAJA
1,SARMIENTO POSADA PEDRO,2,1979-04-19,1983-05-23,0
2,MARTOS AYBAR RAMON,2,1979-04-19,1983-05-23,0
3,BLANES PANIAGUA MARIANO,2,1979-04-19,1983-05-24,0
4,MIRAS CARRASCO JOSE,2,1979-04-19,1983-05-23,1
10,GONZALEZ MARTINEZ JUAN,3,1979-04-19,1987-06-30,0
...,...,...,...,...,...
8758,JUAN IGNACIO MARTINEZ FERNANDEZ,2,2015-06-13,2019-06-15,0
8759,LAURA CRESPI LAZARO,2,2015-06-13,2019-06-15,0
8760,VICTOR MANUEL MANSO OCIO,2,2015-06-13,2019-06-15,0
8761,JOSE MARIA SAENZ BLANCO,4,2007-06-16,2019-06-15,0
